In [25]:
import numpy as np
from numpy import sin, cos, exp, sqrt
import matplotlib.pyplot as plt

# Reloading the module
import importlib

import Solid
importlib.reload(Solid)

import LinearSystem
importlib.reload(LinearSystem)

import Visualization
importlib.reload(Visualization)

import Utility
importlib.reload(Utility)

import Mesh
importlib.reload(Mesh)

import Stress
importlib.reload(Stress)


<module 'Stress' from 'C:\\Users\\bai_f\\Documents\\Timken\\research\\FEM\\3D Elasticity\\Stress.py'>

In [26]:
solid = Solid.Solid('cylinder_external_load')
solid.set_ElasticProperty(E=1, nu=0.3)

In [27]:
func_u = np.empty(3, dtype=object)
func_f = np.empty(3, dtype=object)
func_p = np.empty(3, dtype=object)

func_u[0] = lambda x, y, z: 0
func_u[1] = lambda x, y, z: 0
func_u[2] = lambda x, y, z: 0
func_f[0] = lambda x, y, z: 0
func_f[1] = lambda x, y, z: 0
func_f[2] = lambda x, y, z: 0


# Radius of the cylinder
R = 4
# Half height of the cylinder
Hhalf = 5

# Contact ellipse parameters
a = 0.1
b = 2

# Parameters of the elliptical contact range
phi0 = a / R
s0 = b
# Mid point of the outer surface
rm = R
zm = 0
# Pressure magnitude
Q = 1
p0 = - 3 * Q / R / (2*np.pi * phi0 * s0) 


def func_pr(x, y, z, vect_norm): 

    temp = x + 1j * y
    r = np.linalg.norm(temp)
    phi = np.angle(temp)
        
    s = np.sqrt((r-rm)**2 + (z-zm)**2) # This is unnecessary for a vertical cylindrical surface, kept for generalization

    term_in_sqrt = 1 - (phi/phi0)**2 - (s/s0)**2
    
    if term_in_sqrt>=0:
        return p0 * np.sqrt(term_in_sqrt) 
    else:
        return 0

        
func_p[0] = func_pr
func_p[1] = lambda x, y, z, vect_norm: 0
func_p[2] = lambda x, y, z, vect_norm: 0



In [28]:
# FEM
import LinearSystem
importlib.reload(LinearSystem)
LHS, RHS = LinearSystem.set_LinearSystem(solid, func_u, func_f, func_p)
LinearSystem.solv_LinearSystem(LHS, RHS, solid)

residue 1.0685896612017132e-15


In [29]:
import Stress
importlib.reload(Stress)
Stress.cal_Stress(solid)

In [30]:
import Mesh
importlib.reload(Mesh)
Mesh.output_VTK(solid, solid.name)